In [ ]:
import pandas as pd
import seaborn as sns
import os
import analysis

In [ ]:
analyser = analysis.Analyser("k6")
all_data = analyser.get_data()
all_data

In [ ]:
req_duration_data = all_data[all_data["metric_name"] == "http_req_duration"]
req_duration_data.columns

In [ ]:
data = req_duration_data.copy()
parts = data["name"].str.split("?", expand=True).rename(columns={0: "path", 1: "query"})
parts["path"] = parts["path"].str.lstrip("https://127.0.0.1:8000")
req_duration_data["path"] = parts["path"]
req_duration_data["query"] = parts["query"]

In [ ]:
ignore_vars = [
    "metric_name",
    "metric_value",
    "service",
    "subproto",
    "proto",
    "scenario",
    "status",
    "tls_version",
    "url",
    "name",
    "extra_tags",
    "metadata",
    "check",
    "error",
    "error_code",
    "expected_response",
    "group",
    "method",
    "path",
    "query",
]

In [ ]:
data = req_duration_data.copy()

var, invariant_vars = analysis.condense_vars(
    data, ignore_vars
)
data["vars"] = var

txids = data["query"].str.split("=", expand=True)[1]
data["txids"]=txids
txid_counts = data.groupby(["txids", "vars"]).size()
txid_counts = pd.DataFrame(txid_counts)
txid_counts.rename(columns={0:"committed_count"}, inplace=True)

ax = sns.displot(kind="ecdf", data=txid_counts, x="committed_count", hue="vars")
ax.set(title="number of commit checks before committed")
filename = f"commit_checks_ecdf-committed_count-None-None-vars"
ax.savefig(os.path.join(analyser.plot_dir(), f"{filename}.png"))

In [ ]:
data = req_duration_data.copy()

var, invariant_vars = analysis.condense_vars(
    data, ignore_vars
)
data["vars"] = var

grouped = data.groupby(["path", "query", "vars"])

starts = grouped.min("start_ms")
ends = grouped.max("end_ms")

commit_latency_ms = ends["end_ms"]-starts["start_ms"]

latencies = pd.DataFrame(commit_latency_ms)
latencies.rename(columns={0:"commit_latency"}, inplace=True)

ax = sns.displot(kind="ecdf", data=latencies, x="commit_latency", hue="vars")
ax.set(title="commit latency")
filename = f"commit_latency_ecdf-commit_latency-None-None-vars"
ax.savefig(os.path.join(analyser.plot_dir(), f"{filename}.png"))

In [ ]:
plot_data = req_duration_data.copy(deep=False)
plot_data = plot_data[plot_data["path"] == "v3/kv/put"]
analyser.plot_scatter(plot_data, col="nodes", ignore_vars=ignore_vars)

In [ ]:
plot_data = req_duration_data.copy(deep=False)
analyser.plot_ecdf(plot_data, col="nodes", ignore_vars=ignore_vars)

In [ ]:
plot_data = req_duration_data.copy(deep=False)
analyser.plot_achieved_throughput_bar(plot_data, col="nodes", ignore_vars=ignore_vars)

In [ ]:
plot_data = req_duration_data.copy(deep=False)
analyser.plot_throughput_bar(plot_data, col="nodes", ignore_vars=ignore_vars)

In [ ]:
plot_data = req_duration_data.copy(deep=False)
analyser.plot_target_throughput_latency_line(
    plot_data, col="nodes", ignore_vars=ignore_vars
)